In [1]:
import findspark
#findspark.init('/home/sdmohant/temp/spark-3.0.0-preview2-bin-hadoop2.7')
#findspark.init('/tmp/spark-3.0.0-preview2-bin-hadoop2.7')
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

import pyspark
import random

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName("Affiliations").getOrCreate()

/home/sdmohant/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [5]:
sc = spark.sparkContext

In [6]:
sqlContext = SQLContext(sc)

In [7]:
from pyspark.sql.functions import isnan, when, count, col

# Affiliations

In [8]:
txt_file1 = sc.textFile("../Data/graph/2018-02-02/Affiliations.txt")

In [9]:
header1 = txt_file1.first()
header1

u'154099455\t8223\tshandong university\tShandong University\t86381\t2184963'

In [10]:
fields1 = [StructField(field_name, StringType(),True) for field_name in header1.split("\\t") ]

In [11]:
fields1

[StructField(154099455	8223	shandong university	Shandong University	86381	2184963,StringType,true)]

In [12]:
schema1 = StructType(fields1)

In [13]:
schema1

StructType(List(StructField(154099455	8223	shandong university	Shandong University	86381	2184963,StringType,true)))

In [14]:
#txt_file = txt_file.filter(lambda line : line!= header)

In [15]:
txt_file1.take(10)

[u'154099455\t8223\tshandong university\tShandong University\t86381\t2184963',
 u'94975175\t10705\tdes moines university\tDes Moines University\t1197\t25564',
 u'194999787\t10520\tcancer council queensland\tCancer Council Queensland\t603\t21295',
 u'37712293\t12144\tmadeira interactive technologies institute\tMadeira Interactive Technologies Institute\t413\t3160',
 u'1336114048\t11738\tamerican wind energy association\tAmerican Wind Energy Association\t18\t224',
 u'189460059\t10990\tel bosque university\tEl Bosque University\t2070\t14126',
 u'165644332\t12733\tjalpaiguri government engineering college\tJalpaiguri Government Engineering College\t177\t915',
 u'76806421\t11294\tselex sistemi integrati\tSELEX Sistemi Integrati\t756\t6941',
 u'158506100\t13796\tpennsylvania board of probation and parole\tPennsylvania Board of Probation and Parole\t4\t52',
 u'178498160\t11730\tamerican university in bulgaria\tAmerican University in Bulgaria\t345\t1206']

In [16]:
temp_var1 = txt_file1.map(lambda k: k.split("\t"))

In [17]:
df1 =  temp_var1.toDF()

In [18]:
df1.show()

+----------+-----+--------------------+--------------------+-----+-------+
|        _1|   _2|                  _3|                  _4|   _5|     _6|
+----------+-----+--------------------+--------------------+-----+-------+
| 154099455| 8223| shandong university| Shandong University|86381|2184963|
|  94975175|10705|des moines univer...|Des Moines Univer...| 1197|  25564|
| 194999787|10520|cancer council qu...|Cancer Council Qu...|  603|  21295|
|  37712293|12144|madeira interacti...|Madeira Interacti...|  413|   3160|
|1336114048|11738|american wind ene...|American Wind Ene...|   18|    224|
| 189460059|10990|el bosque university|El Bosque University| 2070|  14126|
| 165644332|12733|jalpaiguri govern...|Jalpaiguri Govern...|  177|    915|
|  76806421|11294|selex sistemi int...|SELEX Sistemi Int...|  756|   6941|
| 158506100|13796|pennsylvania boar...|Pennsylvania Boar...|    4|     52|
| 178498160|11730|american universi...|American Universi...|  345|   1206|
| 205665586|11794|rajendr

In [19]:
df1.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)



In [20]:
df1 = df1.withColumn("AffiliationId",df1["_1"].cast(LongType()))

In [21]:
df1 = df1.withColumn("AffiliationRank",df1["_2"].cast(IntegerType()))

In [22]:
df1 = df1.withColumn("AfNormalizedName",df1["_3"].cast(StringType()))

In [23]:
df1 = df1.withColumn("AfDisplayName",df1["_4"].cast(StringType()))

In [24]:
df1 = df1.withColumn("AfPaperCount",df1["_5"].cast(IntegerType()))

In [25]:
df1 = df1.withColumn("AfCitationCount",df1["_6"].cast(IntegerType()))

In [26]:
drop_columns =["_1","_2","_3","_4","_5","_6"]
df1 = df1.drop(*drop_columns)   

In [27]:
df1.show(10)

+-------------+---------------+--------------------+--------------------+------------+---------------+
|AffiliationId|AffiliationRank|    AfNormalizedName|       AfDisplayName|AfPaperCount|AfCitationCount|
+-------------+---------------+--------------------+--------------------+------------+---------------+
|    154099455|           8223| shandong university| Shandong University|       86381|        2184963|
|     94975175|          10705|des moines univer...|Des Moines Univer...|        1197|          25564|
|    194999787|          10520|cancer council qu...|Cancer Council Qu...|         603|          21295|
|     37712293|          12144|madeira interacti...|Madeira Interacti...|         413|           3160|
|   1336114048|          11738|american wind ene...|American Wind Ene...|          18|            224|
|    189460059|          10990|el bosque university|El Bosque University|        2070|          14126|
|    165644332|          12733|jalpaiguri govern...|Jalpaiguri Govern...|

In [28]:
df1.describe().show()

+-------+-------------------+------------------+--------------------+-------------+-----------------+------------------+
|summary|      AffiliationId|   AffiliationRank|    AfNormalizedName|AfDisplayName|     AfPaperCount|   AfCitationCount|
+-------+-------------------+------------------+--------------------+-------------+-----------------+------------------+
|  count|              18720|             18720|               18720|        18720|            18720|             18720|
|   mean| 3.82126712953312E8|11370.384561965811|                null|         null|5946.249839743589|207863.03814102564|
| stddev|4.843919945299792E8| 1683.982535350433|                null|         null|26028.26335478853|1394591.1596571417|
|    min|               4605|              4983|                  3m|           3M|                1|                 0|
|    max|         2785107011|             14334|ştefan cel mare u...|  Škoda Works|           794193|          67159780|
+-------+-------------------+---

In [29]:
df1.count()

18720

In [30]:
df1.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df1.columns]).show()

+-------------+---------------+----------------+-------------+------------+---------------+
|AffiliationId|AffiliationRank|AfNormalizedName|AfDisplayName|AfPaperCount|AfCitationCount|
+-------------+---------------+----------------+-------------+------------+---------------+
|            0|              0|               0|            0|           0|              0|
+-------------+---------------+----------------+-------------+------------+---------------+



In [31]:
#df1.write.csv('Affiliations.csv')

In [32]:
df1.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('Affiliations.csv',header = 'true')

In [33]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)



In [34]:
#df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Affiliations1.csv')

# Author

In [35]:
author_data = sc.textFile("../Data/graph/2018-02-02/Authors.txt")

In [36]:
author_data.take(10)

[u'2627094434\t19753\tp r la pointe\tP.R. La Pointe\t\t1\t1',
 u'2428216690\t19757\tcaroline ashton\tCaroline Ashton\t\t2\t0',
 u'2696760977\t19757\tespacio trading\tEspacio Trading\t\t1\t0',
 u'2648718335\t19757\t\ud558\ud0dc\uc6c5\t\ud558\ud0dc\uc6c5\t\t1\t0',
 u'2136881011\t18252\tgodson r e e ana\tGodson Ana\t9446555\t39\t166',
 u'2763836685\t19705\tfrank p bell\tFrank P. Bell\t125749732\t1\t1',
 u'2114022572\t19757\tjinguo yu\tJinguo Yu\t31712429\t4\t0',
 u'2599226294\t19757\tjoel martin halpern\tJoel Martin Halpern\t\t1\t0',
 u'2110284670\t18437\te j murry\tE. J. Murry\t36258959\t1\t64',
 u'2763070487\t19491\tj g delcros\tDelcros Jg\t154526488\t1\t6']

In [37]:
author_data = author_data.map(lambda k: k.split("\t"))

In [38]:
df_author =  author_data.toDF()

In [39]:
df_author.head()

Row(_1=u'2627094434', _2=u'19753', _3=u'p r la pointe', _4=u'P.R. La Pointe', _5=u'', _6=u'1', _7=u'1')

In [40]:
df_author.show()

+----------+-----+--------------------------+----------------------------+---------+---+---+
|        _1|   _2|                        _3|                          _4|       _5| _6| _7|
+----------+-----+--------------------------+----------------------------+---------+---+---+
|2627094434|19753|             p r la pointe|              P.R. La Pointe|         |  1|  1|
|2428216690|19757|           caroline ashton|             Caroline Ashton|         |  2|  0|
|2696760977|19757|           espacio trading|             Espacio Trading|         |  1|  0|
|2648718335|19757|                    하태웅|                      하태웅|         |  1|  0|
|2136881011|18252|          godson r e e ana|                  Godson Ana|  9446555| 39|166|
|2763836685|19705|              frank p bell|               Frank P. Bell|125749732|  1|  1|
|2114022572|19757|                 jinguo yu|                   Jinguo Yu| 31712429|  4|  0|
|2599226294|19757|       joel martin halpern|         Joel Martin Halpern|  

In [41]:
df_author = df_author.withColumn("AuthorId",df_author["_1"].cast(LongType()))

In [42]:
df_author = df_author.withColumn("AuthorRank",df_author["_2"].cast(IntegerType()))

In [43]:
df_author = df_author.withColumn("AutNormalizedName",df_author["_3"].cast(StringType()))

In [44]:
df_author = df_author.withColumn("AutDisplayName",df_author["_4"].cast(StringType()))

In [45]:
df_author = df_author.withColumn("AutLastAffiliation",df_author["_5"].cast(LongType()))

In [46]:
df_author = df_author.withColumn("AutPaperCount",df_author["_6"].cast(IntegerType()))

In [47]:
df_author = df_author.withColumn("AutCitationCount",df_author["_7"].cast(IntegerType()))

In [48]:
drop_columns =["_1","_2","_3","_4","_5","_6","_7"]
df_author = df_author.drop(*drop_columns)

In [49]:
df_author.head()

Row(AuthorId=2627094434, AuthorRank=19753, AutNormalizedName=u'p r la pointe', AutDisplayName=u'P.R. La Pointe', AutLastAffiliation=None, AutPaperCount=1, AutCitationCount=1)

In [50]:
df_author.show()

+----------+----------+--------------------------+----------------------------+------------------+-------------+----------------+
|  AuthorId|AuthorRank|         AutNormalizedName|              AutDisplayName|AutLastAffiliation|AutPaperCount|AutCitationCount|
+----------+----------+--------------------------+----------------------------+------------------+-------------+----------------+
|2627094434|     19753|             p r la pointe|              P.R. La Pointe|              null|            1|               1|
|2428216690|     19757|           caroline ashton|             Caroline Ashton|              null|            2|               0|
|2696760977|     19757|           espacio trading|             Espacio Trading|              null|            1|               0|
|2648718335|     19757|                    하태웅|                      하태웅|              null|            1|               0|
|2136881011|     18252|          godson r e e ana|                  Godson Ana|           944655

In [55]:
#import pyspark.sql.functions as f

# add an index column
#df_author_sorted = df_author.withColumn('AuthorId', f.monotonically_increasing_id())

# Sort by index and get first 4000 rows
#working_set = df_author_sorted.sort('AuthorId').limit(4000)

In [56]:
#working_set.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in working_set.columns]).show()

In [57]:
df_author.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_author.columns]).show()

+--------+----------+-----------------+--------------+------------------+-------------+----------------+
|AuthorId|AuthorRank|AutNormalizedName|AutDisplayName|AutLastAffiliation|AutPaperCount|AutCitationCount|
+--------+----------+-----------------+--------------+------------------+-------------+----------------+
|       0|         0|                0|             0|         176261158|            0|               0|
+--------+----------+-----------------+--------------+------------------+-------------+----------------+



In [58]:
#df_author = df_author.subtract(working_set)

In [59]:
new_DF_auth = df_author.filter(((df_author["AutLastAffiliation"] != "") | (df_author["AutLastAffiliation"].isNotNull())))

In [62]:
new_DF_auth.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('Author.csv',header = 'true')

## Paper Author Affiliations

In [65]:
pap_auth_affi = sc.textFile("../Data/graph/2018-02-02/PaperAuthorAffiliations.txt")

In [66]:
header3 = pap_auth_affi.first()

In [67]:
print(header3)


2514067917	2165266595	1334877674	1


In [68]:
pap_auth_affi.take(10)

[u'2514067917\t2165266595\t1334877674\t1',
 u'2514067917\t2334513227\t1334877674\t2',
 u'2514067917\t2159674999\t1334877674\t3',
 u'2514067917\t2133135916\t1334877674\t4',
 u'2527695015\t2645743283\t\t1',
 u'2273582415\t546406812\t205783295\t4',
 u'2273582415\t2109773566\t205783295\t3',
 u'2273582415\t2163045884\t205783295\t2',
 u'2273582415\t1898881442\t205783295\t1',
 u'31410518\t2714234592\t\t1']

In [69]:
temp_var = pap_auth_affi.map(lambda k: k.split("\t"))

In [70]:
df_pap_auth_aff = temp_var.toDF()

In [71]:
df_pap_auth_aff.show()

+----------+----------+----------+---+
|        _1|        _2|        _3| _4|
+----------+----------+----------+---+
|2514067917|2165266595|1334877674|  1|
|2514067917|2334513227|1334877674|  2|
|2514067917|2159674999|1334877674|  3|
|2514067917|2133135916|1334877674|  4|
|2527695015|2645743283|          |  1|
|2273582415| 546406812| 205783295|  4|
|2273582415|2109773566| 205783295|  3|
|2273582415|2163045884| 205783295|  2|
|2273582415|1898881442| 205783295|  1|
|  31410518|2714234592|          |  1|
|2196491992|2701130396|          |  2|
|2196491992|2197163443|          |  1|
|  47114329|2314304603|          |  1|
|1963479517|2296113232| 142823887|  1|
|1963479517|2506097535| 134161618|  3|
|1963479517|2167258348| 134161618|  2|
|1479662648|2586177846|          |  1|
|1963479518|2336340137|          |  2|
|1963479518|2143982647|          |  1|
|2521463243|2160003181| 169521973|  8|
+----------+----------+----------+---+
only showing top 20 rows



In [72]:
df_pap_auth_aff = df_pap_auth_aff.withColumn("PaperID",df_pap_auth_aff["_1"].cast(LongType()))

In [73]:
df_pap_auth_aff = df_pap_auth_aff.withColumn("AuthorId",df_pap_auth_aff["_2"].cast(LongType()))

In [74]:
df_pap_auth_aff = df_pap_auth_aff.withColumn("AffiliationId",df_pap_auth_aff["_3"].cast(LongType()))

In [75]:
df_pap_auth_aff = df_pap_auth_aff.withColumn("AuthorSequenceNumber",df_pap_auth_aff["_4"].cast(IntegerType()))

In [76]:
drop_columns =["_1","_2","_3","_4"]
df_pap_auth_aff = df_pap_auth_aff.drop(*drop_columns)

In [78]:
df_pap_auth_aff.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_pap_auth_aff.columns]).show()

+-------+--------+-------------+--------------------+
|PaperID|AuthorId|AffiliationId|AuthorSequenceNumber|
+-------+--------+-------------+--------------------+
|      0|       0|    229764067|                   0|
+-------+--------+-------------+--------------------+



In [79]:
df_pap_aff_new = df_pap_auth_aff.filter(((df_pap_auth_aff["AffiliationId"] != "") | (df_pap_auth_aff["AffiliationId"].isNotNull())))

In [82]:
df_pap_aff_new.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('PaperAuthorAffilitions.csv',header = 'true')

## Paper References

In [83]:
pap_ref = sc.textFile("../Data/graph/2018-02-02/PaperReferences.txt")

In [84]:
temp_var = pap_ref.map(lambda k: k.split("\t"))

In [85]:
df_pap_ref = temp_var.toDF()

In [86]:
df_pap_ref.show(10)

+----------+----------+
|        _1|        _2|
+----------+----------+
|1819117828|1905052134|
|1819117828|1976258239|
|1479662854|2313093857|
|1479662854|2315253629|
|1479662854|2313966818|
|1479662854| 574521895|
|1479662854| 570984619|
|1479662854|2313539732|
|1479662854|1983800547|
|1479662854|2506074975|
+----------+----------+
only showing top 10 rows



In [87]:
df_pap_ref.count()

1173350871

In [88]:
df_pap_ref = df_pap_ref.withColumn("PaperId",df_pap_ref["_1"].cast(LongType()))

In [89]:
df_pap_ref = df_pap_ref.withColumn("PaperRefId",df_pap_ref["_2"].cast(LongType()))

In [90]:
drop_cols=["_1","_2"]
df_pap_ref = df_pap_ref.drop(*drop_cols)

In [91]:
df_pap_ref.show(2)

+----------+----------+
|   PaperId|PaperRefId|
+----------+----------+
|1819117828|1905052134|
|1819117828|1976258239|
+----------+----------+
only showing top 2 rows



In [92]:
df_pap_ref.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_pap_ref.columns]).show()

+-------+----------+
|PaperId|PaperRefId|
+-------+----------+
|      0|         0|
+-------+----------+



In [96]:
Affilitions_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("Affiliations.csv")

In [97]:
Author_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("Author.csv")

In [98]:
PapAuthAffiliations_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("PaperAuthorAffilitions.csv")

In [99]:
Affilitions_df.printSchema()

root
 |-- AffiliationId: long (nullable = true)
 |-- AffiliationRank: integer (nullable = true)
 |-- AfNormalizedName: string (nullable = true)
 |-- AfDisplayName: string (nullable = true)
 |-- AfPaperCount: integer (nullable = true)
 |-- AfCitationCount: integer (nullable = true)



In [100]:
Author_df.printSchema()

root
 |-- AuthorId: long (nullable = true)
 |-- AuthorRank: integer (nullable = true)
 |-- AutNormalizedName: string (nullable = true)
 |-- AutDisplayName: string (nullable = true)
 |-- AutLastAffiliation: long (nullable = true)
 |-- AutPaperCount: integer (nullable = true)
 |-- AutCitationCount: integer (nullable = true)



In [101]:
PapAuthAffiliations_df.printSchema()

root
 |-- PaperID: long (nullable = true)
 |-- AuthorId: long (nullable = true)
 |-- AffiliationId: long (nullable = true)
 |-- AuthorSequenceNumber: integer (nullable = true)

